### 1. Consegna

Per ogni frame nel FrameSet è necessario assegnare un WN synset ai seguenti elementi:

- **Frame name** (nel caso si tratti di una multiword expression, come per esempio 'Religious_belief', disambiguare il termine principale, che in generale è il **sostantivo** se l'espressione è composta da NOUN+ADJ, e il **verbo** se l'espressione è composta da VERB+NOUN; in generale l'elemento fondamentale è individuato come il **reggente dell'espressione**.
- **Frame Elements (FEs)** del frame; e 
- **Lexical Units (LUs)**.

I contesti di disambiguazione possono essere creati utilizzando le definizioni disponibili (sia quella del frame, sia quelle dei FEs), ottenendo `Ctx(w)`, il contesto per FN terms `w`.

Per quanto riguarda il contesto dei sensi presenti in WN è possibile selezionare glosse ed esempi dei sensi, e dei loro rispettivi iponimi e iperonimi, in modo da avere più informazione, ottenendo quindi il contesto di disambiguazione `Ctx(s)`.



### 2. Algoritmi di mapping

Il mapping può essere effettuato utilizzando (almeno) uno fra i due approcci descritti nel seguito.

- **Approccio a bag of words**, e scelta del senso che permette di massimizzare l'intersezione fra i contesti. In questo caso lo score è calcolato come 
  $$
  score(s,w) = |Ctx(s) \cap Ctx(w)|+1
  $$
  sarà selezionato il senso che massimizza lo *score(s,w)*.
  
- **Approccio grafico**. In questo caso si procede con la costruzione di un grafo che contiene tutti i synset associati ai termini in Framenet (FN)

$$
  \text{FN} = w(\text{FEs}) \cup w(\text{LUs})
$$
### 3. Valutazione dell'output del sistema

#Vari import

In [ ]:
import nltk
from pprint import pprint
nltk.download('framenet_v17')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import framenet as fn
from nltk.corpus.reader.framenet import PrettyList
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 

from nltk.corpus import semcor
from xml.dom import minidom
import random
from nltk.corpus.reader.wordnet import Lemma
from operator import itemgetter
from pprint import pprint

import hashlib
import random
from random import randint
from random import seed

"""
# Funzione di estrazione dei Frame da analizzare
from framenet_extractor import getFrameSetForStudent
# Algoritmo di disambiguazione
from disambiguation import *
"""

#FrameNet Extractor

In [33]:
#ritorna gli ID di tutti i frame
def get_frams_IDs():
    return [f.ID for f in fn.frames()]   
#Funzione del Prof per estrarre dei framenet dall'hash del cognome
def getFrameSetForStudent(surname, list_len=5):
   #n° frame di framenet
    nof_frames = len(fn.frames())
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % nof_frames)
    print('\n################################')
    print('\nstudent: ' + surname)
    print('\n################################\n\n')
    framenet_IDs = get_frams_IDs()
    i = 0
    offset = 0 
    seed(1)
    listaFrames = []
    while i < list_len:
        fID = framenet_IDs[(base_idx+offset)%nof_frames]
        f = fn.frame(fID)
        listaFrames.append(fID)
        fNAME = f.name
        offset = randint(0, nof_frames)
        print('\tID: {a:4d}\tframe: {framename}'.format(a=fID, framename=fNAME))
        i += 1         
    return listaFrames

#Funzioni ausiliarie

In [34]:
lemmatizer = WordNetLemmatizer() 
stop_words_list = []

# Ritorna il POS TAG di una parola
def get_wordnet_pos(word):
    treebank_tag = [tag for (word, tag) in nltk.pos_tag(nltk.word_tokenize(word))][0]
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

# Ritorna una lista con le stop words
def get_stop_words() :
    if len(stop_words_list) == 0 :
        f = open("utils/stop_words_FULL.txt", "r")
        for x in f:
            stop_words_list.append(x.replace("\n",""))
    return stop_words_list


# Calcola il numero di elementi comuni di due liste
def num_common_elements_of_lists(list1, list2) :
    common_elements = set(list1) & set(list2)
    return len(common_elements)


# Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
def get_list_of_gains_words(sentence) :
    list_words_lemma = []
    aus_list_words = sentence.split()
    stop_words_list = get_stop_words()

    for w in aus_list_words :
        word = w.lower().replace("'", "")
        word = ''.join(e for e in word if e.isalnum())
        if word not in stop_words_list :
            pos_tag = get_wordnet_pos(w)
            if pos_tag != '' :
                list_words_lemma.append(lemmatizer.lemmatize(word, pos_tag))

    return list_words_lemma


# Ritorna la lista delle parole della signature di un synset : gloss + examples
def get_signature_of_synset(synset) :
    # Gloss
    signature = get_list_of_gains_words(synset.definition())
    # Examples
    for example in synset.examples() :
        list_ex = get_list_of_gains_words(example)
        signature.extend(list_ex)

    return signature

#Disambiguation

In [35]:
# Algoritmo Lesk disambiguatore leggermente modificato che aggiunge 
# alla signature anche quella degli iponimi e degli iperonimi
def disambiguation_algorithm(word, context, pos_syn=None) :
    best_sense = wn.synsets(word, pos=pos_syn)[0]
    max_overlap = 0
    for sense in wn.synsets(word, pos=pos_syn) :
        # Signature (gloss + examples)
        signature_of_sense = get_signature_of_synset(sense)
        # Aggiungo le info degli iponomi
        for hyponym in sense.hyponyms() :
            signature_of_sense.extend(get_signature_of_synset(hyponym))
        # Aggiungo le info degli iperonimi
        for hypernym in sense.hypernyms() :
            signature_of_sense.extend(get_signature_of_synset(hypernym))  
              
        # Overlap con approccio bag of words (come esplicitato nella formula sommiamo 1)
        overlap = num_common_elements_of_lists(context, signature_of_sense) + 1

        # aggiorno overlap con il valore più alto in assoluto
        if overlap > max_overlap :
            max_overlap = overlap
            best_sense = sense

    return best_sense

#Funzioni utili per la restituzione dei vari contesti

In [36]:
def print_separator():
    print('\n______________________________________________________________________________________________________________________\n\n')

# Restituisce il contesto del nome di un frame, o del frame element se il flag è specificato come False
# Dato dal nome + definizione + frame name in relazione
def get_context_of_frame(frame, is_not_frame_element= True):
    context = set()

    context.add(frame.name.lower()) #Nome
    context.update(get_list_of_gains_words(frame.definition)) #Definizione

    # se non è un frame element, recupero le sue relazioni
    if(is_not_frame_element):
      # relazioni di un frame
      rels = fn.frame_relations(frame=frame)
      for r in rels:
          context.add(r.superFrameName.lower())
          context.add(r.subFrameName.lower())

    return context

# Restituisce il contesto di una lexical unit
def get_context_for_lu(lu):
    context = set()
    # faccio lo split sul '.' e prendo la prima parte
    # esempio in drench.v, prendo solo drench e lo aggiungo al contesto
    context.add((lu.name.split('.')[0]).lower()) 
    context.update(get_list_of_gains_words(lu.definition)) #Definizione
    # aggiungiamo pure i lessemi
    for lex in lu.lexemes:
        context.add(lex.name.lower())

    # fn e cod sono diciture che compaiono nella definizione, le rimuoviamo
    if 'fn' in context:
        context.remove('fn')
    if 'cod' in context:
        context.remove('cod')
    return context

# Da un insieme di parole ritorna la parola reggente
def get_reggente_multiwords(words) :
    # rimpiazzo underscore con lo spazio
    words = words.replace("_", " ")
    #strip() toglie gli spazi bianchi all'inizio e alla fine
    words = words.strip()
    # Controllo se è una multiwords. 
    # Es: Verbo + Nome o Nome + Aggettivo
    if len(words.split(" ")) > 1:
        #Pos tagging
        tokens = nltk.word_tokenize(words)
        tags = nltk.pos_tag(tokens)
        
        # Cerco i verbi, in verbs avrò tutti i verbi 
        verbs = [(words, tag) for (words, tag) in nltk.pos_tag(nltk.word_tokenize(words)) if tag.startswith('V')]
        # Se c'è un verbo è il reggente
        if len(verbs) > 0:
            # il metodo prende il primo verbo perchè words è sempre un nome di frame,
            # per cui non posso avere più verbi insieme ma solo uno con aggettivo o sostantivo
            return verbs[0][0]
        else: # altrimenti il reggente è il primo dei nomi
            nouns = [(words, tag) for (words, tag) in nltk.pos_tag(nltk.word_tokenize(words)) if tag.startswith('N')]
            return nouns[0][0]
    else:
       # se invece è una sola parola non c'è bisogno di fare i controlli di prima
        return words


# Main

In [ ]:
############################################################################################

if __name__ == '__main__':
    cognomi = {"BushajAntonino","BushajAldo","Frisullo"}
    
    for c in cognomi:
        list_of_mapped_synset = []
        
        # Per ogni frame restituito da getFrameSetForStudent
        for i in getFrameSetForStudent(c):
           # prendo l'i-esimo frame 
            f = fn.frame(i)
            # Stampo il nome del frame
            print('\033[1m' + "FrameName: " + '\033[0m', f.name)
        
            #Stampo il contesto del frame
            context = get_context_of_frame(f)
            print('\033[1m' + "Ctx: " + '\033[0m', context)
            
            # Determina il synset con algoritmo di disambiguazione e mapping Bag of Words
            best_synset = disambiguation_algorithm(get_reggente_multiwords(f.name), context)
            # stampo le varie informazioni
            print('\033[1m' + "BestSense WN Synset: " + '\033[0m', best_synset)
            print('\033[1m' + "Definizione synset: " + '\033[0m', best_synset.definition())
            list_of_mapped_synset.append(best_synset)
            
            # Stampo la definizione del frame
            print('\033[1m' + "\nDefinizione frame: " + '\033[0m', f.definition)
            
            # In questo ciclo stampo le varie informazioni sui frame elements
            print('\033[1m' + "\nFEs: " + '\033[0m')
            for frame_element_key in f.FE.keys():
                # Recupero il frame element corrente
                frame_element = f.FE[frame_element_key]
                print('\033[1m' + "\tFrame element: " + '\033[0m', frame_element.name)
                # Prendo il contesto del frame element e lo stampo in output
                # il booleano mi indica se sono nel caso in cui sto trattando un frame element o no
                context = get_context_of_frame(frame_element,False)
                print("\t", context)
                # Stampo in output anche la sua definizione
                print('\033[1m' + "\tDefinition: " + '\033[0m',frame_element.definition)
                try:
                    best_synset = disambiguation_algorithm(get_reggente_multiwords(frame_element.name), context)
                    print('\033[1m' + "\n\tBestSense WN Synset: " + '\033[0m', best_synset)
                    print('\033[1m' + "\tDefinizione synset: " + '\033[0m', best_synset.definition())
                    list_of_mapped_synset.append(best_synset)
                except:
                    #wordnet non ha un synset corrispondente a questo frame
                    print('\033[1m' + "\n\tBestSense WN Synset: Nessun risultato " + '\033[0m')
                    list_of_mapped_synset.append("")
                print("\t----------------------------------------------")

            # In questo ciclo stampo le varie informazioni sui lexical units
            print('\033[1m' + "\nLUs: " + '\033[0m')
            for lexical_unit_key in f.lexUnit.keys():
                # Recupero il lexical unit corrente
                lexical_unit = f.lexUnit[lexical_unit_key]
                print('\033[1m' + "\tLexical Unit: " + '\033[0m', lexical_unit.name)
                # Recupero e stampo in output il contesto
                context = get_context_for_lu(lexical_unit)
                print("\t", context)
                try:
                    best_synset = disambiguation_algorithm(get_reggente_multiwords(str(lexical_unit.name).split(".")[0]), context, str(lexical_unit.name).split(".")[1])#Nome, contesto, postag
                    print('\033[1m' + "\n\tBestSense WN Synset: " + '\033[0m', best_synset)
                    print('\033[1m' + "\tDefinizione synset: " + '\033[0m', best_synset.definition())
                    list_of_mapped_synset.append(best_synset)
                except:
                    print('\033[1m' + "\n\tBestSense WN Synset: Nessun risultato " + '\033[0m')
                    list_of_mapped_synset.append("")
                print("\t----------------------------------------------------------------------")

            print_separator()

        # Valutazione risultato tra mapping e annotazione
        annotation_file = open("./annotazione/" + c + "_annotazione.txt", "r")
        i = 0; mapping_corretti = 0

        for line in annotation_file:
            # Lettura del synset annotato
            annotated_synset = line.split(";")[1]
            annotated_synset = annotated_synset.strip()
            # Confronto tra annotazione e mapping
            if str("Synset('"+annotated_synset+"')") == str(list_of_mapped_synset[i]) or (str(annotated_synset) == "" and str(list_of_mapped_synset[i]) == "") :
                mapping_corretti+=1
            i+=1

        print('\033[1m' + "Valutazione : " + '\033[0m', mapping_corretti/len(list_of_mapped_synset))